In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import random
import seaborn as sns
pd.options.display.max_rows=300
pd.set_option('display.max_columns', None)

In [ ]:
df_type = pd.read_csv("../../02.AUMCdb_SAKI_trajCluster/df_mixAK_fea3_C3_aumc.csv")
df_type_filt = df_type.loc[:,["stay_id","groupHPD"]]
df_type_filt = df_type_filt.drop_duplicates()
all_lst = df_type_filt.stay_id.unique().tolist()
print("Number of included patients:",len(all_lst))

df_type_filt_count = pd.DataFrame(df_type_filt["groupHPD"].value_counts()).reset_index()
df_type_filt_count.columns = ["groupHPD","count_all"]
df_type_filt_count

In [3]:
df_time = pd.read_csv("/public/hanl/jupyter_dir/kidney_sepsis_penotype_v3/00.data_aumc/disease_definition//AKI/aumcdb_sk_event_time.csv")
df_time = df_time[["stay_id","sepsis_onset", "saki_onset"]]
df_time = pd.merge(df_type_filt, df_time, how="inner", on="stay_id")
df_time.head(2)

,stay_id,groupHPD,sepsis_onset,saki_onset
0,5,2,0.0,3.0
1,15,2,1.0,3.0


# Diuretic

In [ ]:
df_di = pd.read_csv("/public/hanl/jupyter_dir/database/AMUCdb/clean/medication//AUMCdb_diuretics.csv")
# Unit conversion
df_di.loc[df_di["drug"]=='Bumetanide (Burinex)',"administered"] *= 80
df_di = df_di[['stay_id','starttime', 'endtime', 'administered']]
df_di.columns = ['stay_id', 'starttime', 'endtime','amount']
df_di = df_di[df_di["amount"]>0]

import datetime
def dateHourRange(beginDateHour, endDateHour):
    dhours = []
    dhour = beginDateHour 
    date = beginDateHour
    while date <= endDateHour:
        dhours.append(date)
        dhour = dhour + 61
        date = dhour 
    return dhours

tem_di = df_di
tem_di["time_lst"] = tem_di.apply(lambda x: dateHourRange(x['starttime'],x['endtime']),axis=1)
tem_di["time_num"] = tem_di["time_lst"].map(lambda x: len(x))
tem_di["amount_hourly"] =  tem_di["amount"]/tem_di["time_num"] 

tem_di['idx'] = range(len(tem_di))
tem_add = tem_di.set_index('idx').time_lst.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'diu_starttime'})
tem_di_hourly = pd.merge(tem_di, tem_add,how="inner", on="idx")
tem_di_hourly["diu_endtime"] = tem_di_hourly["diu_starttime"] + 60
tem_di_hourly['diu_endtime'] = tem_di_hourly[['diu_endtime', 'endtime']].min(axis=1) 
tem_di_hourly = tem_di_hourly.loc[:,['stay_id', 'diu_starttime','diu_endtime',  'amount_hourly']] # 'starttime', 'endtime', 'amount',
tem_di_hourly = tem_di_hourly.rename(columns={"amount_hourly":"amount"})
tem_di_hourly.head(2)

In [ ]:
df_di = tem_di_hourly
df_di = df_di[df_di["stay_id"].isin(all_lst)]
df_time_cut = df_time[['stay_id', 'saki_onset']]
df_di = pd.merge(df_di,df_time_cut, how="right",on="stay_id")
df_di = df_di[df_di["diu_starttime"] >= df_di["saki_onset"]]
df_di = df_di.sort_values(["stay_id","diu_starttime"])
df_di = df_di.drop(["saki_onset","diu_endtime"],axis=1)
df_di.columns = ["stay_id","diuretic_time","diuretic_amout"]
df_di["diuretic_time"] = df_di["diuretic_time"] / 60
df_di.head(1)

In [ ]:
len(df_di.stay_id.unique())

# UO

In [ ]:
import datetime
def dateHourRange_v2(beginDateHour, endDateHour):
    beginDateHour= beginDateHour #.strftime('%Y-%m-%d %H:%M:%S')
    endDateHour= endDateHour#.strftime('%Y-%m-%d %H:%M:%S')
    dhours = []
    dhour = endDateHour #datetime.datetime.strptime(endDateHour, "%Y-%m-%d %H:%M:%S")
    date = endDateHour#[:]
    while date > beginDateHour:
        dhours.append(date)
        dhour = dhour - 1 #datetime.timedelta(hours=1)
        date = dhour#.strftime("%Y-%m-%d %H:%M:%S")
    return dhours

In [ ]:
df_uo = pd.read_csv("../../00.data_aumc/feature_data/aumc_icu_feature.csv")
df_uo = df_uo[['stay_id', 'charttime','urineoutput']]
df_uo = df_uo[df_uo["stay_id"].isin(all_lst)]
df_uo = df_uo[~df_uo.urineoutput.isnull()]
df_uo.head(1)

In [ ]:
tmp = df_uo
tmp['last_charttime'] = tmp.groupby('stay_id')['charttime'].shift(1)
tmp["last_charttime"].fillna(tmp["charttime"] - 1, inplace=True)
tmp["last_charttime"] = tmp["last_charttime"].astype("int64")
tmp["time_lst"] = tmp.apply(lambda x: dateHourRange_v2(x['last_charttime'],x['charttime']),axis=1)
tmp["time_num"] = tmp["time_lst"].map(lambda x: len(x))
tmp["urineoutput_hourly"] =  tmp["urineoutput"]/tmp["time_num"] 

tmp['idx'] = range(len(tmp))
tem_add = tmp.set_index('idx').time_lst.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'uo_charttime_hourly'})
tmp_hourly = pd.merge(tmp, tem_add,how="inner", on="idx")
df_uo_hourly = tmp_hourly[["stay_id","uo_charttime_hourly","urineoutput_hourly"]]
df_uo_hourly = df_uo_hourly.sort_values(["stay_id","uo_charttime_hourly"])
df_uo_hourly.head()

# Diuretic reactivity

In [ ]:
df_merge = pd.merge(df_di, df_uo_hourly, how="left", on="stay_id")
print(df_merge.shape)
df_merge=df_merge[(df_merge["uo_charttime_hourly"]<=df_merge["diuretic_time"]+24) & (df_merge["uo_charttime_hourly"]>=df_merge["diuretic_time"]-24)]
print(df_merge.shape)

df_merge["uo_charttime_hourly_b"] = df_merge["uo_charttime_hourly"] - 1
time_win = 2
df_merge["diuretic_time_b"] = df_merge["diuretic_time"] - time_win  
df_merge["diuretic_time_a"] = df_merge["diuretic_time"] + time_win  
df_merge.head()

In [ ]:
# the urine volume 2 hours before the diuretic is used
df_bt = df_merge  
df_bt['intersection'] = df_bt.apply(lambda row: max(min(row['uo_charttime_hourly'], row['diuretic_time']) - max(row['uo_charttime_hourly_b'], row['diuretic_time_b']),0),axis=1)
df_bt["urineoutput_before_useDiu"] = df_bt["urineoutput_hourly"] * df_bt["intersection"]  
df_bt = df_bt.groupby(["stay_id","diuretic_time","diuretic_amout"]).agg({"urineoutput_before_useDiu":"sum"}).reset_index()
df_bt = pd.merge(df_di,df_bt,how="left",on=["stay_id","diuretic_time","diuretic_amout"])
df_bt = df_bt.fillna(0)
df_bt = df_bt.drop_duplicates()
df_bt.to_csv("df_bt.csv",index=False)
df_bt.head()

In [ ]:
df_bt.shape,df_bt[df_bt["urineoutput_before_useDiu"]<200].shape

In [ ]:
# the urine volume 2 hours after the diuretic is used
df_at = df_merge  
df_at['intersection'] = df_at.apply(lambda row: max(min(row['uo_charttime_hourly'], row['diuretic_time_a']) - max(row['uo_charttime_hourly_b'], row['diuretic_time']),0),axis=1)
df_at["urineoutput_after_useDiu"] = df_at["urineoutput_hourly"] * df_at["intersection"] 
df_at = df_at.groupby(["stay_id","diuretic_time","diuretic_amout"]).agg({"urineoutput_after_useDiu":"sum"}).reset_index()
df_at = pd.merge(df_di,df_at,how="left",on=["stay_id","diuretic_time","diuretic_amout"])
df_at = df_at.fillna(0)
df_at = df_at.drop_duplicates()
df_at.to_csv("df_at.csv",index=False)
df_at.head()

In [ ]:
df_at.shape,df_at[df_at["urineoutput_after_useDiu"]<200].shape

In [ ]:
df_bt = pd.read_csv("df_bt.csv")
df_at = pd.read_csv("df_at.csv")
change_percent = 0.1
df_bt_at = pd.merge(df_bt, df_at, how="inner",on=["stay_id","diuretic_time","diuretic_amout"])
df_bt_at.loc[(df_bt_at["urineoutput_after_useDiu"] <= df_bt_at["urineoutput_before_useDiu"] + df_bt_at["urineoutput_before_useDiu"]*change_percent) | (df_bt_at["urineoutput_after_useDiu"] <=200),"one_label_diu_res"] = "Non-responsive"
df_bt_at.loc[((df_bt_at["urineoutput_after_useDiu"] > 200) & (df_bt_at["urineoutput_before_useDiu"]>200)) | ((df_bt_at["urineoutput_after_useDiu"] > df_bt_at["urineoutput_before_useDiu"] + df_bt_at["urineoutput_before_useDiu"]*change_percent) & (df_bt_at["urineoutput_after_useDiu"] >200)),"one_label_diu_res"] = "responsive"
df_bt_at.to_csv("tmp_df_diuretic_responsitive.csv",index=False)

In [ ]:
df_stat = df_bt_at.groupby("stay_id").agg({'one_label_diu_res': lambda x: list(x)}).reset_index()

def process_list(lst):
    if len(lst) > 2:
        counts = {}
        for item in lst:
            counts[item] = counts.get(item, 0) + 1
        max_count = max(counts.values())
        most_common = [k for k, v in counts.items() if v == max_count]
        if len(most_common) > 1:
            return most_common[1]
        else:
            return most_common[0]
    elif len(lst) == 2:
        return lst[-1]
    else:
        return lst[0]

df_stat['label_diu_res'] = df_stat['one_label_diu_res'].apply(process_list)
df_stat.head()

In [ ]:
tmp = pd.merge(df_bt_at,df_stat[['stay_id', 'label_diu_res']],how="inner",on="stay_id")
df_bt_at = tmp[["stay_id","label_diu_res"]].drop_duplicates()

In [ ]:
df_sur = pd.read_csv("../../02.AUMCdb_SAKI_trajCluster/sk_survival.csv")
df_sur = df_sur[['stay_id','groupHPD', 'mortality_28d', 'survival_28day']]
df_bt_at = pd.merge(df_bt_at,df_sur, how="right",on="stay_id")
df_bt_at["label_diu_res"] = df_bt_at["label_diu_res"].fillna("No diuretic")
df_bt_at.head(1)

In [ ]:
df_bt_at.to_csv("df_diuretic_FST.csv",index=False)

# merge

In [3]:
df_bt_at = pd.read_csv("df_diuretic_FST.csv")

result = df_bt_at.groupby(["groupHPD", "label_diu_res"])["stay_id"].agg(["count"]).reset_index()
df_count_all = pd.DataFrame(df_bt_at.groupby("groupHPD")["stay_id"].agg("count")).reset_index()
df_count_all.columns = ["groupHPD","groupHPD_count"]

result = pd.merge(result,df_count_all,how="inner",on="groupHPD")
result["count_percentage"] = result["count"] / result["groupHPD_count"] *100
result

,groupHPD,label_diu_res,count,groupHPD_count,count_percentage
0,1,No diuretic,119,564,21.099291
1,1,Non-responsive,138,564,24.468085
2,1,responsive,307,564,54.432624
3,2,No diuretic,445,1400,31.785714
4,2,Non-responsive,176,1400,12.571429
5,2,responsive,779,1400,55.642857
6,3,No diuretic,76,219,34.703196
7,3,Non-responsive,78,219,35.616438
8,3,responsive,65,219,29.680365


In [21]:
df_fea = pd.read_csv("./aumcdb_diuretic_survival.csv")


In [22]:
df_fea = pd.read_csv("./aumcdb_diuretic_survival.csv")
df_fea = df_fea[df_fea["time"]==1]
df_fea = df_fea[['stay_id',  'creatinine', 'urineoutput','baseline_Scr',
                 'colloid_bolus','sofa_norenal',"Congestive_heart_failure",
                  'weight', 'age', 'gender', 'height', 
               
       'respiration_sofa', 'cardiovascular_sofa', 'aniongap', 'pt', 'sbp',
    'coagulation_sofa', 'cum_fluidbance', 'is_diu',
       'baseexcess',   'hemoglobin',
         'liver_sofa',
       'furosemide_amount', 'fluidbance', 'gcs_verbal', 'gcs', 'mbp',
       'potassium', 'spo2', 'platelet', 'renal_sofa', 'gcs_eyes',
       'bicarbonate', 'sodium', 'temperature', 'gcs_motor', 'po2', 
       'ptt', 'glucose', 'pco2', 'chloride', 'rbc', 'wbc',
       'ph', 'sofa', 'mchc',  'mcv', 'hematocrit',
       'first_use_amount', 'cns_sofa',  'time',
       'heart_rate', 'fio2',  'resp_rate',  
       'furosemide_amount_kg', 'dbp', 'mch', 'is_diu_byid', 'lactate', 'rdw',
       'calcium', 'first_use_time' 
              ]]
df_bt_at_ = pd.merge(df_bt_at,df_fea, how="right",on="stay_id")
df_bt_at_["gcs"] = df_bt_at_["gcs"].fillna(df_bt_at_["gcs"].mean())
df_bt_at_["platelet"] = df_bt_at_["platelet"].fillna(df_bt_at_["platelet"].mean())
df_bt_at_["rbc"] = df_bt_at_["rbc"].fillna(df_bt_at_["rbc"].mean())
df_bt_at_["mch"] = df_bt_at_["mch"].fillna(df_bt_at_["mch"].mean())
df_bt_at_["chloride"] = df_bt_at_["chloride"].fillna(df_bt_at_["chloride"].mean())
df_bt_at_["mcv"] = df_bt_at_["mcv"].fillna(df_bt_at_["mcv"].mean())
df_bt_at_["mchc"] = df_bt_at_["mchc"].fillna(df_bt_at_["mchc"].mean())


df_bt_at_.to_csv("df_diuretic_responsitive.csv",index=False)
df_bt_at_.label_diu_res.value_counts()

responsive        1151
No diuretic        640
Non-responsive     392
Name: label_diu_res, dtype: int64